In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,  explained_variance_score, r2_score


In [2]:
train_price = train['SalePrice']

In [3]:
data = pd.concat((train, test), sort =False).reset_index(drop=True)

In [4]:
data.loc[:, 'PoolQC'] = data.loc[:, 'PoolQC'].fillna('No')
data.loc[:, 'MiscFeature'] = data.loc[:, 'MiscFeature'].fillna('No')
data.loc[:, 'Fence'] = data.loc[:, 'Fence'].fillna('No')
data.loc[:, 'GarageCond'] = data.loc[:, 'GarageCond'].fillna('No')
data.loc[:, 'GarageQual'] = data.loc[:, 'GarageQual'].fillna('No')
data.loc[:, 'GarageFinish'] = data.loc[:, 'GarageFinish'].fillna('No')
data.loc[:, 'GarageYrBlt'] = data.loc[:, 'GarageYrBlt'].fillna(0)
data.loc[:, 'GarageType'] = data.loc[:, 'GarageType'].fillna('No')
data.loc[:, 'GarageType'] = data.loc[:, 'GarageType'].fillna('No')
data.loc[:, 'FireplaceQu'] = data.loc[:, 'FireplaceQu'].fillna('No')
data.loc[:, 'Electrical'] = data.loc[:, 'Electrical'].fillna('SBrkr')
data.loc[:, 'BsmtFinType2'] = data.loc[:, 'BsmtFinType2'].fillna('No')
data.loc[:, 'BsmtFinType1'] = data.loc[:, 'BsmtFinType1'].fillna('No')
data.loc[:, 'BsmtExposure'] = data.loc[:, 'BsmtExposure'].fillna('None')
data.loc[:, 'BsmtCond'] = data.loc[:, 'BsmtCond'].fillna('No')
data.loc[:, 'BsmtQual'] = data.loc[:, 'BsmtQual'].fillna('No')
data.loc[:, 'MasVnrArea'] = data.loc[:, 'MasVnrArea'].fillna(0)
data.loc[:, 'MasVnrType'] = data.loc[:, 'MasVnrType'].fillna('None')
data.loc[:, 'Alley'] = data.loc[:, 'Alley'].fillna('No')
data.loc[:, 'LotFrontage'] = data.loc[:, 'LotFrontage'].fillna(0)
data.loc[:, 'Utilities'] = data.loc[:, 'Utilities'].fillna('ELO')
data.loc[:, 'LotFrontage'] = data.loc[:, 'LotFrontage'].fillna(0)
data.loc[:, 'BsmtFinSF1'] = data.loc[:, 'BsmtFinSF1'].fillna(0)
data.loc[:, 'BsmtFinSF2'] = data.loc[:, 'BsmtFinSF2'].fillna(0)
data.loc[:, 'BsmtUnfSF'] = data.loc[:, 'BsmtUnfSF'].fillna(0)
data.loc[:, 'TotalBsmtSF'] = data.loc[:, 'TotalBsmtSF'].fillna(0)
data.loc[:, 'BsmtFullBath'] = data.loc[:, 'BsmtFullBath'].fillna(0)
data.loc[:, 'KitchenQual'] = data.loc[:, 'KitchenQual'].fillna('Po')
data.loc[:, 'GarageCars'] = data.loc[:, 'GarageCars'].fillna(0)
data.loc[:, 'GarageArea'] = data.loc[:, 'GarageArea'].fillna(0)
data.loc[:, 'BsmtHalfBath'] = data.loc[:, 'BsmtHalfBath'].fillna(0)

In [5]:
data = data.replace({"Alley" : {'No' : 0,"Grvl" : 1, "Pave" : 2}})
data = data.replace({"LotShape" : {"IR3" : 1, "IR2" : 2, 'IR1' : 3, 'Reg' : 4}})
data = data.replace({'LandCountour' : {'Low' : 1, 'HLS' : 2, 'Bnk' : 3, 'Lvl' : 4}})
data = data.replace({'Utilities' : {'ELO' : 1, 'NoSeWa' : 2, 'NoSewr' : 3, 'AllPub' : 4}})
data = data.replace({'LandSlope' : {'Sev' : 1, 'Mod' : 2, 'Gtl' : 3}})
data = data.replace({'ExtrQual' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'ExtrCond' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'BsmtQual' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'BsmtCond' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'BsmtExposure' : {'None' : 0, 'No' : 1, 'Mn' : 2, 'Av' : 3, 'Gd' : 4}})
data = data.replace({'BsmtFinType1' : {'No' : 0, 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6}})
data = data.replace({'BsmtFinType2' : {'No' : 0, 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6}})
data = data.replace({'HeatingQC' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'KitchenQual' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'Functional' : {'Sal' : 1, 'Sev' : 2, 'Maj2' : 3, 'Maj1' : 4, 'Mod' : 5, 'Min2' : 6, 'Min1' : 7}})
data = data.replace({'FireplaceQu' : {'No' : 0 ,'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'GarageFinish' : {'No' : 0 ,'Unf' : 1, 'RFn' : 2, 'Fin' : 3}})
data = data.replace({'GarageQual' : {'No' : 0 ,'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'GarageCond' : {'No' : 0 ,'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}})
data = data.replace({'PavedDrive' : {'N' : 1, 'P' : 2, 'Y' : 3}})
data = data.replace({'PoolQC' : {'No' : 0, 'Fa' : 1, 'TA' : 2, 'Gd' : 3, 'Ex' : 4}})
data = data.replace({'Fence' : {'No' : 0, 'MnWw' : 1, 'GdWo' : 2, 'MnPrv' : 3, 'GdPrv' : 4}})

In [6]:
objects = data.columns[data.dtypes == 'object']
ints = data.columns[data.dtypes == 'Int64']

In [7]:
droplist = ['YrSold', 'MoSold', 'MiscVal', 'PoolArea',
            'ScreenPorch', '3SsnPorch', 'EnclosedPorch', 'KitchenAbvGr', 'BedroomAbvGr', 'BsmtHalfBath',
           'LowQualFinSF', 'BsmtFinSF2', 'OverallCond', 'MSSubClass', 'Id', 'Fence', 'Utilities',
            'LandSlope', 'BsmtFinType2', 'PoolQC', 'LotShape', 'Alley']

In [8]:
data.drop(droplist, axis=1, inplace=True)

In [9]:
ints = data.columns[data.dtypes == 'Int64']

In [10]:
from scipy.stats import skew
import numpy as np
x= data[ints]

In [11]:
data['LotArea'] = np.log(data['LotArea'])
print(skew(data['LotArea']))

data['SalePrice'] = np.log(data['SalePrice'])
print(skew(data['SalePrice']))

-0.5052823792296689
nan


In [12]:
data[ints]

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,1stFlrSF,2ndFlrSF,...,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF
0,9.041922,7,2003,2003,4,1,6,5,856,854,...,4,8,0,0,2,3,3,3,0,61
1,9.169518,6,1976,1976,4,4,5,5,1262,0,...,3,6,1,3,2,3,3,3,298,0
2,9.328123,7,2001,2002,4,2,6,5,920,866,...,4,6,1,3,2,3,3,3,0,42
3,9.164296,7,1915,1970,3,1,5,4,961,756,...,4,7,1,4,1,3,3,3,0,35
4,9.565214,8,2000,2000,4,3,6,5,1145,1053,...,4,9,1,3,2,3,3,3,192,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,7.568379,4,1970,1970,3,1,1,4,546,546,...,3,5,0,0,0,0,0,3,0,0
2915,7.546446,4,1970,1970,3,1,3,3,546,546,...,3,6,0,0,1,3,3,3,0,24
2916,9.903488,5,1960,1996,3,1,5,5,1224,0,...,3,7,1,3,1,3,3,3,474,0
2917,9.253496,5,1992,1992,4,3,6,3,970,0,...,3,6,0,0,0,0,0,3,80,32


In [13]:
data.drop(['SalePrice'], axis = 1, inplace = True)

In [14]:
sc = StandardScaler()
data[ints] = sc.fit_transform(data[ints])

In [15]:
data[ints]

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,1stFlrSF,2ndFlrSF,...,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF
0,-0.103678,0.646183,1.046258,0.896833,0.577094,-0.583117,1.163184,0.885619,-0.773861,1.207379,...,0.738229,0.986849,-0.924311,-0.978830,0.316551,0.278091,0.268343,0.31503,-0.740760,0.200006
1,0.146571,-0.063185,0.154764,-0.395604,0.577094,2.221033,0.690033,0.885619,0.261075,-0.785025,...,-0.768687,-0.287758,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,1.614879,-0.702843
2,0.457635,0.646183,0.980221,0.848965,0.577094,0.351600,1.163184,0.885619,-0.610718,1.235375,...,0.738229,-0.287758,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,-0.740760,-0.081209
3,0.136329,0.646183,-1.859351,-0.682812,-0.527521,-0.583117,0.690033,-0.158453,-0.506205,0.978742,...,0.738229,0.349546,0.623632,1.235629,-0.798060,0.278091,0.268343,0.31503,-0.740760,-0.184815
4,0.922629,1.355551,0.947203,0.753229,0.577094,1.286316,1.163184,0.885619,-0.037170,1.671651,...,0.738229,1.624153,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,0.776967,0.540424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,-2.993667,-1.481920,-0.043346,-0.682812,-0.527521,-0.583117,-1.202573,-0.158453,-1.564083,0.488807,...,-0.768687,-0.925062,-0.924311,-0.978830,-1.912672,-3.913374,-3.942917,0.31503,-0.740760,-0.702843
2915,-3.036683,-1.481920,-0.043346,-0.682812,-0.527521,-0.583117,-0.256270,-1.202525,-1.564083,0.488807,...,-0.768687,-0.287758,-0.924311,-0.978830,-0.798060,0.278091,0.268343,0.31503,-0.740760,-0.347624
2916,1.586070,-0.772552,-0.373528,0.561757,-0.527521,-0.583117,0.690033,0.885619,0.164209,-0.785025,...,-0.768687,0.349546,0.623632,0.682014,-0.798060,0.278091,0.268343,0.31503,3.006130,-0.702843
2917,0.311272,-0.772552,0.683057,0.370284,0.577094,1.286316,1.163184,-1.202525,-0.483263,-0.785025,...,-0.768687,-0.287758,-0.924311,-0.978830,-1.912672,-3.913374,-3.942917,0.31503,-0.108374,-0.229217


In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
train = data[ints][:1460]
test = data[ints][1460:]

In [18]:
test

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,1stFlrSF,2ndFlrSF,...,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF
1460,0.521438,-0.772552,-0.340510,-1.113625,-0.527521,-0.583117,-0.256270,-1.202525,-0.671897,-0.785025,...,-0.768687,-0.925062,-0.924311,-0.978830,-0.798060,0.278091,0.268343,0.31503,0.365916,-0.702843
1461,0.923592,-0.063185,-0.439565,-1.257229,-0.527521,-0.583117,0.690033,-1.202525,0.431865,-0.785025,...,0.738229,-0.287758,-0.924311,-0.978830,-0.798060,0.278091,0.268343,0.31503,2.365838,-0.170014
1462,0.862579,-0.772552,0.848148,0.657493,0.577094,-0.583117,1.163184,-0.158453,-0.590326,0.850426,...,-0.768687,-0.287758,0.623632,0.682014,1.431163,0.278091,0.268343,0.31503,0.935064,-0.199616
1463,0.222314,-0.063185,0.881166,0.657493,-0.527521,-0.583117,1.163184,0.885619,-0.595424,0.796766,...,0.738229,0.349546,0.623632,1.235629,1.431163,0.278091,0.268343,0.31503,2.104979,-0.170014
1464,-1.130843,1.355551,0.683057,0.370284,0.577094,-0.583117,0.690033,0.885619,0.306959,-0.785025,...,0.738229,-0.925062,-0.924311,-0.978830,0.316551,0.278091,0.268343,0.31503,-0.740760,0.510823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,-2.993667,-1.481920,-0.043346,-0.682812,-0.527521,-0.583117,-1.202573,-0.158453,-1.564083,0.488807,...,-0.768687,-0.925062,-0.924311,-0.978830,-1.912672,-3.913374,-3.942917,0.31503,-0.740760,-0.702843
2915,-3.036683,-1.481920,-0.043346,-0.682812,-0.527521,-0.583117,-0.256270,-1.202525,-1.564083,0.488807,...,-0.768687,-0.287758,-0.924311,-0.978830,-0.798060,0.278091,0.268343,0.31503,-0.740760,-0.347624
2916,1.586070,-0.772552,-0.373528,0.561757,-0.527521,-0.583117,0.690033,0.885619,0.164209,-0.785025,...,-0.768687,0.349546,0.623632,0.682014,-0.798060,0.278091,0.268343,0.31503,3.006130,-0.702843
2917,0.311272,-0.772552,0.683057,0.370284,0.577094,1.286316,1.163184,-1.202525,-0.483263,-0.785025,...,-0.768687,-0.287758,-0.924311,-0.978830,-1.912672,-3.913374,-3.942917,0.31503,-0.108374,-0.229217


In [19]:
train

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,1stFlrSF,2ndFlrSF,...,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF
0,-0.103678,0.646183,1.046258,0.896833,0.577094,-0.583117,1.163184,0.885619,-0.773861,1.207379,...,0.738229,0.986849,-0.924311,-0.978830,0.316551,0.278091,0.268343,0.31503,-0.740760,0.200006
1,0.146571,-0.063185,0.154764,-0.395604,0.577094,2.221033,0.690033,0.885619,0.261075,-0.785025,...,-0.768687,-0.287758,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,1.614879,-0.702843
2,0.457635,0.646183,0.980221,0.848965,0.577094,0.351600,1.163184,0.885619,-0.610718,1.235375,...,0.738229,-0.287758,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,-0.740760,-0.081209
3,0.136329,0.646183,-1.859351,-0.682812,-0.527521,-0.583117,0.690033,-0.158453,-0.506205,0.978742,...,0.738229,0.349546,0.623632,1.235629,-0.798060,0.278091,0.268343,0.31503,-0.740760,-0.184815
4,0.922629,1.355551,0.947203,0.753229,0.577094,1.286316,1.163184,0.885619,-0.037170,1.671651,...,0.738229,1.624153,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,0.776967,0.540424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.231462,-0.063185,0.914184,0.753229,0.577094,-0.583117,-1.202573,0.885619,-0.526598,0.834095,...,-0.768687,0.349546,0.623632,0.682014,0.316551,0.278091,0.268343,0.31503,-0.740760,-0.110811
1456,0.767421,-0.063185,0.220801,0.178812,0.577094,-0.583117,0.690033,-1.202525,2.328397,-0.785025,...,-0.768687,0.349546,2.171575,0.682014,-0.798060,0.278091,0.268343,0.31503,2.018026,-0.702843
1457,0.029126,0.646183,-1.000876,1.040437,-0.527521,-0.583117,1.163184,0.885619,0.072441,1.902620,...,0.738229,1.624153,2.171575,1.235629,0.316551,0.278091,0.268343,0.31503,-0.740760,0.185205
1458,0.170329,-0.772552,-0.703711,0.561757,-0.527521,0.351600,1.163184,-0.158453,-0.207960,-0.785025,...,0.738229,-0.925062,-0.924311,-0.978830,-0.798060,0.278091,0.268343,0.31503,2.152408,-0.702843


In [20]:
X_tr = train
Y_tr = np.log(train_price)
X_te = test

In [21]:
null = Y_tr.isnull().sum()
print(null[null > 0])

[]


In [22]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [23]:
data_dmatrix = xgb.DMatrix(data=X_tr,label=Y_tr)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_tr, Y_tr, test_size=0.2, random_state=123)

In [40]:
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'num_boost_round': [10, 20, 30, 40, 50],
             'num_estimators' : [100, 200, 300, 500]}

In [26]:
from sklearn.model_selection import GridSearchCV

In [41]:
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

In [42]:
xgb_grid.fit(X_train,y_train)

Fitting 2 folds for each of 180 candidates, totalling 360 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  70 tasks      | elapsed:    1.8s
[Parallel(n_jobs=5)]: Done 360 out of 360 | elapsed:   11.1s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
             iid='warn', n_jobs=5,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.03, 0.05, 0.07],
                         'max_depth': [5, 6, 7], 'min_child_weight': [4],
                         'nthread': [4],
                         'num_boost_round': [10, 20, 30, 40, 50],

In [43]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.8600249024810355
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 7, 'min_child_weight': 4, 'nthread': 4, 'num_boost_round': 10, 'num_estimators': 100, 'objective': 'reg:squarederror', 'silent': 1, 'subsample': 0.7}


In [55]:
data_XGB = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.03, max_depth = 7, min_child_weight = 4,
                           n_estimators = 500, nthread = 4, objective = 'reg:squarederror', silent = 1, subsample = 0.7, num_boost_round = 20)
data_XGB.fit(X_tr, Y_tr)
pred_XGB = data_XGB.predict(X_te)

In [56]:
pred_XGB = np.exp(pred_XGB)

In [57]:
len(pred_XGB)

1459

In [58]:
testest = {'SalePrice' : pred_XGB, 'Id' : range(1461, 2920)}
ultra = pd.DataFrame(testest, columns = ['Id', 'SalePrice'])

In [59]:
ultra

,Id,SalePrice
0,1461,118375.578125
1,1462,162850.593750
2,1463,183212.078125
3,1464,191066.078125
4,1465,190130.703125
...,...,...
1454,2915,75627.468750
1455,2916,81855.968750
1456,2917,154771.062500
1457,2918,114966.953125


In [35]:
#data_RFR = GradientBoostingRegressor(n_estimators = 180, max_features = 'sqrt')
#data_RFR.fit(X_train, Y_train)
#pred_RFR = data_RFR.predict(X_test)

In [36]:
#pred_RFR = np.exp(pred_RFR)

In [37]:
#testest = {'SalePrice' : pred_GB, 'Id' : range(1461, 2920)}
#ultra = pd.DataFrame(testest, columns = ['Id', 'SalePrice'])

In [38]:
#ultra

In [60]:
ultra.to_csv (r'C:\Users\Lenovo\Documents\submission_15.csv', index = False, header=True)